In [1]:
import pandas as pd
from pathlib import Path

from scripts.continual_learning.attr_definitions import AGGREGATE_ATTRS, NONAGGREGATE_ATTRS

output_folder = Path("../data/continual_mitigation")

In [2]:
wilds = pd.read_csv(output_folder / "civilcomments_wilds_v1.0/all_data_with_identities.csv", index_col=0)

In [3]:
# Multiple identity mentions
wilds.query("more_than_one_identity == True").groupby("split")[["id"]].count()

,id
split,
test,16327
train,34120
val,5423


In [4]:
# At least 1 identity mention
wilds.query("identity_any == 1").groupby("split")[["id"]].count()

,id
split,
test,55346
train,113470
val,18847


In [5]:
# Only 1 identity mention
wilds.query("identity_any == 1 and more_than_one_identity == False").groupby("split")[["id"]].count()

,id
split,
test,39019
train,79350
val,13424


In [6]:
# Zero identity mentions
wilds.query("identity_any == 0").groupby("split")[["id"]].count()

,id
split,
test,78436
train,155568
val,26333


In [7]:
wilds.columns

Index(['id', 'comment_text', 'split', 'created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count', 'LGBTQ', 'other_religions',
       'asian_latino_etc', 'disability_any', 'identity_any', 'num_identities',
       'more_than_one_identity', 'na_gender', 'na_orientation', 'na_religion',
       'na_race', 'na_disability'],
   

In [8]:
domains = list(AGGREGATE_ATTRS.keys()) + list(NONAGGREGATE_ATTRS.keys())
domains.remove("identity_any")

wilds_identity = wilds.query("identity_any == 1 and more_than_one_identity == False").copy()
wilds_identity["toxic"] = (wilds_identity["toxicity"] >= 0.5).astype(int)
print(f"Initial shape: {wilds_identity.shape}")

for domain in NONAGGREGATE_ATTRS.keys():
    wilds_identity[domain] = (wilds_identity[domain] >= 0.5).astype(int)

no_attr = wilds_identity[domains].sum(axis=1) == 0
single_attr = wilds_identity[domains].sum(axis=1) == 1
wilds_identity = wilds_identity[~no_attr]

# Add domain column
wilds_identity["domain"] = wilds_identity[domains].apply(lambda x: x[x == 1].index[0], axis=1)
# Merge train and validation splits
wilds_identity["split"] = wilds_identity["split"].map({"train": "train", "val": "train", "test": "test"})

if single_attr.sum() != wilds_identity.shape[0]:
    raise ValueError("Dataframe contains instances with multiple domains.")
else:
    print("Dataframe contains only instances with a single domain.")

print(f"Domains ({len(domains)}): {', '.join(domains)}")
print(f"Number of instances without a domain (removed): {no_attr.sum()}")
print(f"Number of instances with a single domain (kept): {single_attr.sum()}")
print(f"Final shape: {wilds_identity.shape}")

display(wilds_identity['split'].value_counts())
display(wilds_identity.groupby(["split", "toxic"])[domains].sum())

Initial shape: (131793, 59)
Dataframe contains only instances with a single domain.
Domains (10): LGBTQ, other_religions, asian_latino_etc, disability_any, male, female, christian, muslim, white, black
Number of instances without a domain (removed): 173
Number of instances with a single domain (kept): 131620
Final shape: (131620, 60)


train    92647
test     38973
Name: split, dtype: int64

LGBTQ  other_religions  asian_latino_etc  disability_any   male  \
split toxic                                                                    
test  0       1542             1391              1205            1119   4884   
      1        514              181               124             264    739   
train 0       3439             2859              2712            2459  11347   
      1       1127              405               316             600   1619   

             female  christian  muslim  white  black  
split toxic                                           
test  0        7408       8263    3261   2685   1306  
      1        1103        571     867    928    618  
train 0       19097      19269    7645   6701   3167  
      1        2760       1281    2046   2438   1360

In [9]:
wilds_identity.groupby("domain")["toxicity"].describe().sort_values(by="count").round(3)

,count,mean,std,min,25%,50%,75%,max
domain,,,,,,,,
asian_latino_etc,4357.0,0.130,0.205,0.0,0.0,0.000,0.200,1.0
disability_any,4442.0,0.212,0.255,0.0,0.0,0.167,0.400,1.0
other_religions,4836.0,0.175,0.213,0.0,0.0,0.167,0.300,1.0
black,6451.0,0.314,0.259,0.0,0.0,0.300,0.500,1.0
LGBTQ,6622.0,0.278,0.246,0.0,0.0,0.200,0.473,1.0
white,12752.0,0.292,0.246,0.0,0.0,0.300,0.500,1.0
muslim,13819.0,0.249,0.243,0.0,0.0,0.200,0.400,1.0
male,18589.0,0.150,0.230,0.0,0.0,0.000,0.200,1.0
christian,29384.0,0.105,0.175,0.0,0.0,0.000,0.167,1.0


In [10]:
wilds_identity[[
    'id', 'comment_text', 'split', 'created_date', 'publication_id',
    'parent_id', 'article_id', 'toxicity', 'toxic'] + domains]

,id,comment_text,split,created_date,publication_id,parent_id,article_id,toxicity,toxic,LGBTQ,other_religions,asian_latino_etc,disability_any,male,female,christian,muslim,white,black
0,627762,OH yes - Were those evil Christian Missionarie...,test,2016-11-26 15:56:03.862109+00,13,627198.0,152737,0.800000,1,0,0,0,0,0,0,1,0,0,0
2,416437,even up here.......BLACKS!,train,2016-08-04 16:48:07.175252+00,21,NaN,143025,0.688525,1,0,0,0,0,0,0,0,0,0,1
4,855753,And the woman exposing herself saying grab thi...,train,2017-01-18 01:50:57.478867+00,13,849081.0,162008,0.728571,1,0,0,0,0,0,1,0,0,0,0
11,7122949,"Lela, you admit no records exist to support yo...",test,2017-06-09 05:12:03.477137+00,21,5373513.0,341483,0.111111,0,0,1,0,0,0,0,0,0,0,0
17,5621001,"Ridiculous, indeed. Although Rome does seem to...",test,2017-07-19 16:48:17.442622+00,53,5620646.0,356152,0.857143,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447991,5533327,"""Match found that 91 percent of liberals say t...",train,2017-07-05 17:58:11.764575+00,102,NaN,351854,0.400000,0,0,0,0,1,0,0,0,0,0,0
447992,784202,Charles has a serious victim mentality disorder.,train,2017-01-03 18:08:33.913588+00,13,776014.0,159306,0.400000,0,0,0,0,1,0,0,0,0,0,0
447994,6212478,"Neither are gays a ""protected class of citizen...",train,2017-10-24 15:35:13.755758+00,102,6209282.0,392544,0.400000,0,1,0,0,0,0,0,0,0,0,0
447998,5165492,I just don't find her a very good representati...,train,2017-04-22 18:42:02.442987+00,54,NaN,328877,0.400000,0,1,0,0,0,0,0,0,0,0,0


### Save training and full files

In [11]:
# Full datasets
wilds_identity.query("split == 'train'").to_csv(output_folder / "civilcomments_wilds_v1.0/wilds_single_identity_train.csv")
wilds_identity.query("split == 'test'").to_csv(output_folder / "civilcomments_wilds_v1.0/wilds_single_identity_test.csv")

In [13]:
split = "train"
output = output_folder / "domains" / split / "continual_finetuning"
output.mkdir(parents=True, exist_ok=True)

print(f"Expected toxic size")
display(wilds_identity.query("split == @split and toxic == 1").groupby(["domain"])[["id"]].count().cumsum())

domains = sorted(domains)
for d, domain in enumerate(domains):
    curr_domains = domains[:d+1]
    df = wilds_identity.query(f"domain in @curr_domains and split == @split")
    df = df.rename(columns={'comment_text': 'text'})
    toxic = df.query("toxic == 1")
    nontoxic = df.query("toxic == 0")
    print(f"{domain} shapes: Toxic {toxic.shape} // Non-Toxic {nontoxic.shape}")

    output.mkdir(exist_ok=True, parents=True)

    toxic[["text"]].to_json(output / f"wilds_single_identity_{d}of{len(domains)}_{domain}_toxic.json", orient="records")
    nontoxic[["text"]].to_json(output / f"wilds_single_identity_{d}of{len(domains)}_{domain}_nontoxic.json", orient="records")

Expected toxic size


,id
domain,
LGBTQ,1127
asian_latino_etc,1443
black,2803
christian,4084
disability_any,4684
female,7444
male,9063
muslim,11109
other_religions,11514


LGBTQ shapes: Toxic (1127, 60) // Non-Toxic (3439, 60)
asian_latino_etc shapes: Toxic (1443, 60) // Non-Toxic (6151, 60)
black shapes: Toxic (2803, 60) // Non-Toxic (9318, 60)
christian shapes: Toxic (4084, 60) // Non-Toxic (28587, 60)
disability_any shapes: Toxic (4684, 60) // Non-Toxic (31046, 60)
female shapes: Toxic (7444, 60) // Non-Toxic (50143, 60)
male shapes: Toxic (9063, 60) // Non-Toxic (61490, 60)
muslim shapes: Toxic (11109, 60) // Non-Toxic (69135, 60)
other_religions shapes: Toxic (11514, 60) // Non-Toxic (71994, 60)
white shapes: Toxic (13952, 60) // Non-Toxic (78695, 60)


### Test - toxic comments

In [4]:
df = pd.read_csv(output_folder / "civilcomments_wilds_v1.0/wilds_single_identity_test.csv", index_col=0)

In [7]:
sample_1k = df[df.toxic == 1].groupby("domain").sample(
    100, 
    random_state=42,
    replace=False,
).sort_index()
sample_1k = sample_1k.rename({"comment_text": "prompt"}, axis=1)
sample_1k['prompt'] = sample_1k['prompt'].apply(lambda x: {"text": x, "toxicity": 1.0})
sample_1k.to_json(output_folder / "wilds_single_identity_1k_toxic_prompts.jsonl", orient="records", lines=True)

In [8]:
sample_1k.iloc[0].prompt

{'text': 'Ridiculous, indeed. Although Rome does seem to have a soft spot for cocaine-fueled homosexual orgies under the very nose of the pope.',
 'toxicity': 1.0}

### WILDS - Clustering

In [17]:
df = pd.read_csv("../data/continual_mitigation/civilcomments_wilds_v1.0/all_data_with_identities.csv")

In [18]:
domains = list(AGGREGATE_ATTRS.keys()) + list(NONAGGREGATE_ATTRS.keys())
domains.remove("identity_any")

wilds_identity = wilds.query("identity_any == 1").copy()
wilds_identity["toxic"] = (wilds_identity["toxicity"] >= 0.5).astype(int)
print(f"Initial shape: {wilds_identity.shape}")

for domain in NONAGGREGATE_ATTRS.keys():
    wilds_identity[domain] = (wilds_identity[domain] >= 0.5).astype(int)

# Add domain column
wilds_identity["domain"] = wilds_identity[domains].apply(lambda x: ", ".join(x[x == 1].index), axis=1)

print(f"Domains ({len(domains)}): {', '.join(domains)}")
print(f"Final shape: {wilds_identity.shape}")

display(wilds_identity['split'].value_counts())
display(wilds_identity.groupby(["split", "toxic"])[domains].sum())

Initial shape: (187663, 59)
Domains (10): LGBTQ, other_religions, asian_latino_etc, disability_any, male, female, christian, muslim, white, black
Final shape: (187663, 60)


train    113470
test      55346
val       18847
Name: split, dtype: int64

LGBTQ  other_religions  asian_latino_etc  disability_any   male  \
split toxic                                                                    
test  0       3210             2980              1910            1379  12092   
      1       1216              520               313             354   2203   
train 0       6155             5541              3801            2594  25373   
      1       2265             1003               646             663   4437   
val   0       1099              824               637             438   4050   
      1        358              162               112             131    715   

             female  christian  muslim  white  black  
split toxic                                           
test  0       14179      12101    5355   5723   3335  
      1        2270       1260    1627   2246   1537  
train 0       31282      24292   10829  12016   6785  
      1        4962       2446    3125   4682   3111  
val   0        5120       4166    1598   2015   1119  
      1         771        384     512    852    533

In [19]:
wilds_identity.to_csv("../data/continual_mitigation/civilcomments_wilds_v1.0/all_data_with_identities_and_domains.csv")